In [153]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

load_dotenv()
API_KEY = os.getenv('API_KEY')

client = OpenAI(api_key=API_KEY)

In [154]:
df = pd.read_csv('data/sample.csv')

# add column "summaries", "theme" and "keywords" to df
df['summaries'] = None
df['theme'] = None
df['keywords'] = None
df['note'] = None

In [155]:
from pydantic import BaseModel
class AbstractSummary(BaseModel):
    keywords: list[str]
    summaries: list[str]
    note: str

prompt = """
Extract key findings from the abstract as bullet points, following these rules:

Formatting:
- Present only substantiated findings
- Skip bullet points entirely if the abstract is purely theoretical or does not present findings
- Limit to only the most important findings (typically 0-3 bullets)

Each bullet must:
- State a complete, specific conclusion (e.g., "AI chatbots foster creative collaboration by enabling anonymous idea sharing")
- Be independently understandable without context
- Use concise, precise, simple language
- Focus solely on verified results and findings
- Include metrics when available, but not required
- Include ONLY findings where AI is directly involved in the result
- Describes an AI-related element, explicitly name it as such (e.g. "Design guidelines" → "AI design guidelines", "Explanations" → "AI explanations", "The system's outputs..." → "The AI system's outputs...")
- Express each finding in Subject-Predicate-Object (SPO) format where possible

Do NOT include:
- Research methodology or process descriptions
- References to "this paper" or "this study"
- Vague comparisons without explanation
- Framework descriptions or conceptual models
- Hypotheses or future work

Sample bullets:
❌ "The system showed improved performance over baseline"
✅ "Virtual reality environments enable deeper emotional engagement in therapy sessions"
✅ "AI-assisted writing tools reduce cognitive load by managing document structure"

Output Format:
- Summaries: Clear, quantified bullet points [Follow rules from first section]
- Note [if no bullets extracted]: state the paper type, e.g.
  - "Workshop announcement"
  - "Conceptual framework"
  - "Design methodology"
  - "Technical specification"
  - "Systematic review"
  
In addition, extract 1-3 keywords from the abstract:
- Keywords [1-3]: Main topics and themes of the paper

include:
- Domain terms (e.g., "Healthcare", "Education")
- Target outcomes (e.g., "Team Efficiency", "Learning")
- Specific contexts (e.g., "Emergency Response", "K-12")

DO NOT include:
- Generic HCI/AI-related terms (e.g., "Human-AI Interaction", "Human-Computer Interaction", "Artificial Intellegence")
"""

def get_summary(abstract):
    chat_completion = client.beta.chat.completions.parse(
        messages=[
            {
                "role": "user",
                "content": prompt + " The following is the abstract: " + abstract,
            },
        ],
        model="gpt-4o",
        response_format=AbstractSummary,
        temperature=0.5
    )
    
    return chat_completion.choices[0].message.parsed

In [156]:
# len(df)

for i in range(21,30):
    print(i, df.iloc[i]['title'])
    s = get_summary(df.iloc[i]['abstract'])
    df.at[i, 'summaries'] = s.summaries # [{"summary": summary.bullet, "effect": summary.effect} for summary in s.summaries]
    df.at[i, 'keywords'] = s.keywords
    df.at[i, 'note'] = s.note
    

21 Effects of Communication Directionality and AI Agent Differences in Human-AI Interaction
22 UX Research on Conversational Human-AI Interaction: A Literature Review of the ACM Digital Library
23 Human-AI interaction and ethics of AI: how well are we following the guidelines
24 Forward Reasoning Decision Support: Toward a More Complete View of the Human-AI Interaction Design Space
25 Understanding User Perceptions, Collaborative Experience and User Engagement in Different Human-AI Interaction Designs for Co-Creative Systems
26 Towards Mutual Theory of Mind in Human-AI Interaction: How Language Reflects What Students Perceive About a Virtual Teaching Assistant
27 Re-examining Whether, Why, and How Human-AI Interaction Is Uniquely Difficult to Design
28 The Human in the Infinite Loop: A Case Study on Revealing and Explaining Human-AI Interaction Loop Failures
29 Towards fluid software architectures: bidirectional human-AI interaction


In [157]:
def print_summary(summary):
    # print(f"Theme: {summary.theme}")
    print()
    print(summary.title)
    print(f"Keywords: {summary.keywords}")
    print(f"Note: {summary.note}")
    for s in summary.summaries:
        print(f"- {s}")
        
for i in range(21,30):
    print_summary(df.iloc[i])


Effects of Communication Directionality and AI Agent Differences in Human-AI Interaction
Keywords: ['Human-AI Collaboration', 'Social Perception', 'Cooperative Games']
Note: 
- AI agents perceived as more responsive improve social rapport in cooperative games.
- Direction of communication impacts perceived intelligence of AI partners.
- Bias against AI varies with communication direction and agent type.

UX Research on Conversational Human-AI Interaction: A Literature Review of the ACM Digital Library
Keywords: ['Conversational Agents', 'Human-Human Interaction', 'User Experience']
Note: 
- Designing with social boundaries, such as privacy, disclosure, and identification, is crucial for ethical polyadic conversational agents.

Human-AI interaction and ethics of AI: how well are we following the guidelines
Keywords: ['AI Technology Acceptance', 'Human-AI Interaction', 'Ethics Guidelines']
Note: 
- HAI interactions designed according to HAI interaction guidelines do not necessarily alig